## Веб-скрейпинг из Википедии и анализ данных на примере страницы с президентами в США

<img src='web_scrap-696x400.jpg' style='width:600px;height:350px'/>

Компания "Тралала" решила проверить навыки работников и поручила одному из них:

1. Сделать веб-скрейпинг с любой веб-страницы(в данном примере - страницы из Википедии);
2. Провести необходимые преобразования данных и убрать лишний 'мусор', который засоряет данные, чтобы можно было их анализировать и проводить стастические действия.
3. С помощью SQLite3 сделать контент-анализ данных и вычислить некоторые статистические признаки.

###### Импортируем необходимые библиотеки

In [496]:
import pandas as pd
import numpy as np
import sqlite3

Для начала найдем подходящую нам страницу.

Выберем для анализа таблицу с президентами США - https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States

<img src='srcinshot_1.png' style='width:700px;height:400px'/>

Привяжем ссылку к переменной url:

In [497]:
url = 'https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States'

Далее воспользуемся Pandas для того чтобы прочитать страницу и передать ссылку в переменную tables

In [498]:
tables = pd.read_html(url)

Посмотрим сколько таблиц в было на странице:

In [499]:
len(tables)

6

Нужная таблица с презедентами находиться под индексом 0

<img src='scrinshot_2.png' style='width:650px;height:400px'/>

Сохраним таблицу в переменную df_USA_presidents.

In [500]:
df_USA_presidents = tables[0]

Посмотрим на получившеюся таблицу:

In [501]:
df_USA_presidents.head()

,No.[a],Portrait,Name(Birth–Death),Term[14],Party[b][15],Party[b][15].1,Election,Vice President[16]
0,1,NaN,George Washington(1732–1799)[17],"April 30, 1789–March 4, 1797",NaN,Unaffiliated,1788–1789 1792,John Adams[c]
1,2,NaN,John Adams(1735–1826)[19],"March 4, 1797–March 4, 1801",NaN,Federalist,1796,Thomas Jefferson[d]
2,3,NaN,Thomas Jefferson(1743–1826)[21],"March 4, 1801–March 4, 1809",NaN,Democratic-Republican,1800 1804,Aaron Burr George Clinton
3,4,NaN,James Madison(1751–1836)[22],"March 4, 1809–March 4, 1817",NaN,Democratic-Republican,1808 1812,"George Clinton[e] Vacant afterApril 20, 1812 E..."
4,5,NaN,James Monroe(1758–1831)[24],"March 4, 1817–March 4, 1825",NaN,Democratic-Republican,1816 1820,Daniel D. Tompkins


Получившаяся таблица не особо годиться для использования, так что давайте займемся подготовкой данных к анализу.

Можем заметить что в колонках `Portrait` и `Party[b][15]` у нас проставлены NaN значения. В итоговой таблице нам не понадобяться портреты президентов и цвет их партии, так что применем функцию drop для данных колонок.

In [502]:
df_USA_presidents = df_USA_presidents.drop(['Portrait', 'Party[b][15]'], axis=1)

In [503]:
df_USA_presidents.head()

,No.[a],Name(Birth–Death),Term[14],Party[b][15].1,Election,Vice President[16]
0,1,George Washington(1732–1799)[17],"April 30, 1789–March 4, 1797",Unaffiliated,1788–1789 1792,John Adams[c]
1,2,John Adams(1735–1826)[19],"March 4, 1797–March 4, 1801",Federalist,1796,Thomas Jefferson[d]
2,3,Thomas Jefferson(1743–1826)[21],"March 4, 1801–March 4, 1809",Democratic-Republican,1800 1804,Aaron Burr George Clinton
3,4,James Madison(1751–1836)[22],"March 4, 1809–March 4, 1817",Democratic-Republican,1808 1812,"George Clinton[e] Vacant afterApril 20, 1812 E..."
4,5,James Monroe(1758–1831)[24],"March 4, 1817–March 4, 1825",Democratic-Republican,1816 1820,Daniel D. Tompkins


Следующим шагом предлагаю редактировать название колонок, чтобы избавиться от сносок, которые были полезны на html странице, но здесь не имеют ничего подобного.

In [504]:
df_USA_presidents = df_USA_presidents.rename(columns={'No.[a]': 'No', 
                                                      'Term[14]': 'Term', 
                                                      'Party[b][15].1': 'Party', 
                                                      'Vice President[16]': 'Vice President'})

In [505]:
df_USA_presidents.head()

,No,Name(Birth–Death),Term,Party,Election,Vice President
0,1,George Washington(1732–1799)[17],"April 30, 1789–March 4, 1797",Unaffiliated,1788–1789 1792,John Adams[c]
1,2,John Adams(1735–1826)[19],"March 4, 1797–March 4, 1801",Federalist,1796,Thomas Jefferson[d]
2,3,Thomas Jefferson(1743–1826)[21],"March 4, 1801–March 4, 1809",Democratic-Republican,1800 1804,Aaron Burr George Clinton
3,4,James Madison(1751–1836)[22],"March 4, 1809–March 4, 1817",Democratic-Republican,1808 1812,"George Clinton[e] Vacant afterApril 20, 1812 E..."
4,5,James Monroe(1758–1831)[24],"March 4, 1817–March 4, 1825",Democratic-Republican,1816 1820,Daniel D. Tompkins


Поскольку колонка `No` является номерной от первого президент до настоящего. Сделаем эту колонку индексом в нашем датафрейме.

In [506]:
df_USA_presidents = df_USA_presidents.set_index('No')

In [507]:
df_USA_presidents.head()

,Name(Birth–Death),Term,Party,Election,Vice President
No,,,,,
1,George Washington(1732–1799)[17],"April 30, 1789–March 4, 1797",Unaffiliated,1788–1789 1792,John Adams[c]
2,John Adams(1735–1826)[19],"March 4, 1797–March 4, 1801",Federalist,1796,Thomas Jefferson[d]
3,Thomas Jefferson(1743–1826)[21],"March 4, 1801–March 4, 1809",Democratic-Republican,1800 1804,Aaron Burr George Clinton
4,James Madison(1751–1836)[22],"March 4, 1809–March 4, 1817",Democratic-Republican,1808 1812,"George Clinton[e] Vacant afterApril 20, 1812 E..."
5,James Monroe(1758–1831)[24],"March 4, 1817–March 4, 1825",Democratic-Republican,1816 1820,Daniel D. Tompkins


Колонка с именем и датой рождения/смерти не совсем удобна для того, чтобы проводить анализ. Следующим шагом разделим их отдельно на колонку Name и для начала Birth_Death.

In [508]:
df_USA_presidents['Name'] = df_USA_presidents['Name(Birth–Death)'].apply(lambda x: x.split('(')[0]) 

df_USA_presidents['Birth–Death'] = df_USA_presidents['Name(Birth–Death)'].\
                                                     apply(lambda x: x.split('(')[1].split(')[')[0])



In [509]:
df_USA_presidents.head()

,Name(Birth–Death),Term,Party,Election,Vice President,Name,Birth–Death
No,,,,,,,
1,George Washington(1732–1799)[17],"April 30, 1789–March 4, 1797",Unaffiliated,1788–1789 1792,John Adams[c],George Washington,1732–1799
2,John Adams(1735–1826)[19],"March 4, 1797–March 4, 1801",Federalist,1796,Thomas Jefferson[d],John Adams,1735–1826
3,Thomas Jefferson(1743–1826)[21],"March 4, 1801–March 4, 1809",Democratic-Republican,1800 1804,Aaron Burr George Clinton,Thomas Jefferson,1743–1826
4,James Madison(1751–1836)[22],"March 4, 1809–March 4, 1817",Democratic-Republican,1808 1812,"George Clinton[e] Vacant afterApril 20, 1812 E...",James Madison,1751–1836
5,James Monroe(1758–1831)[24],"March 4, 1817–March 4, 1825",Democratic-Republican,1816 1820,Daniel D. Tompkins,James Monroe,1758–1831


In [510]:
df_USA_presidents.tail()

,Name(Birth–Death),Term,Party,Election,Vice President,Name,Birth–Death
No,,,,,,,
42,Bill Clinton(b. 1946)[71],"January 20, 1993–January 20, 2001",Democratic,1992 1996,Al Gore,Bill Clinton,b. 1946
43,George W. Bush(b. 1946)[72],"January 20, 2001–January 20, 2009",Republican,2000 2004,Dick Cheney,George W. Bush,b. 1946
44,Barack Obama(b. 1961)[73],"January 20, 2009–January 20, 2017",Democratic,2008 2012,Joe Biden,Barack Obama,b. 1961
45,Donald Trump(b. 1946)[74],"January 20, 2017–January 20, 2021",Republican,2016,Mike Pence,Donald Trump,b. 1946
46,Joe Biden(b. 1942)[6],"January 20, 2021–Incumbent",Democratic,2020,Kamala Harris,Joe Biden,b. 1942


Как мы можем заметить колонка `Birth–Death` содержит ныне живущих людей, что были на должности президента или сейчас на ней без даты смерти. Чтобы в запросах корректно отображалась информация, запишем для таких президентов значения - 0 и уведомим в характеристиках датафрейма, что в колонке Death_date значение 0 - это отсутствие даты смерти.

In [511]:
df_USA_presidents['Birth_date'] = df_USA_presidents['Birth–Death'].\
                                                apply(lambda x: x.split('–')[0] if '–' in x else x.split(' ')[1])

df_USA_presidents['Death_date'] = df_USA_presidents['Birth–Death'].\
                                                apply(lambda x: x.split('–')[1] if '–' in x else 0)

In [512]:
df_USA_presidents.head()

,Name(Birth–Death),Term,Party,Election,Vice President,Name,Birth–Death,Birth_date,Death_date
No,,,,,,,,,
1,George Washington(1732–1799)[17],"April 30, 1789–March 4, 1797",Unaffiliated,1788–1789 1792,John Adams[c],George Washington,1732–1799,1732,1799
2,John Adams(1735–1826)[19],"March 4, 1797–March 4, 1801",Federalist,1796,Thomas Jefferson[d],John Adams,1735–1826,1735,1826
3,Thomas Jefferson(1743–1826)[21],"March 4, 1801–March 4, 1809",Democratic-Republican,1800 1804,Aaron Burr George Clinton,Thomas Jefferson,1743–1826,1743,1826
4,James Madison(1751–1836)[22],"March 4, 1809–March 4, 1817",Democratic-Republican,1808 1812,"George Clinton[e] Vacant afterApril 20, 1812 E...",James Madison,1751–1836,1751,1836
5,James Monroe(1758–1831)[24],"March 4, 1817–March 4, 1825",Democratic-Republican,1816 1820,Daniel D. Tompkins,James Monroe,1758–1831,1758,1831


Преобразование колонок мы сделали, теперь удалим исходные, чтобы не было лишних. 

In [513]:
df_USA_presidents = df_USA_presidents.drop(['Name(Birth–Death)', 'Birth–Death'], axis=1)

Следующим шагом переставим колонки так, как было изначально.

In [514]:
df_USA_presidents = df_USA_presidents[['Name', 'Birth_date', 'Death_date', 'Term', 'Party', 'Election', 'Vice President']]

In [515]:
df_USA_presidents.head()

,Name,Birth_date,Death_date,Term,Party,Election,Vice President
No,,,,,,,
1,George Washington,1732,1799,"April 30, 1789–March 4, 1797",Unaffiliated,1788–1789 1792,John Adams[c]
2,John Adams,1735,1826,"March 4, 1797–March 4, 1801",Federalist,1796,Thomas Jefferson[d]
3,Thomas Jefferson,1743,1826,"March 4, 1801–March 4, 1809",Democratic-Republican,1800 1804,Aaron Burr George Clinton
4,James Madison,1751,1836,"March 4, 1809–March 4, 1817",Democratic-Republican,1808 1812,"George Clinton[e] Vacant afterApril 20, 1812 E..."
5,James Monroe,1758,1831,"March 4, 1817–March 4, 1825",Democratic-Republican,1816 1820,Daniel D. Tompkins


Данные выглядят уже намного лучше, но все еще требуют исправлений. Займемся сроком управления и преобразуем даты в количество дней, для того чтобы было удобно в дальнейшем, если понадобиться, распределять президентов по этому признаку. По скольку нынешний президент все еще на посту, можно было бы использовать сегодняшнюю дату, но ввиду возможных вычислений, лучше будет убрать его и работать только с предыдущими президентами.

In [516]:
df_USA_presidents['start'] = df_USA_presidents['Term'].apply(lambda date: date.split('–')[0])
df_USA_presidents['end'] = df_USA_presidents['Term'].apply(lambda date: date.split('–')[1])

df_USA_presidents = df_USA_presidents[df_USA_presidents['Name'] != 'Joe Biden']

In [517]:
df_USA_presidents

,Name,Birth_date,Death_date,Term,Party,Election,Vice President,start,end
No,,,,,,,,,
1,George Washington,1732,1799,"April 30, 1789–March 4, 1797",Unaffiliated,1788–1789 1792,John Adams[c],"April 30, 1789","March 4, 1797"
2,John Adams,1735,1826,"March 4, 1797–March 4, 1801",Federalist,1796,Thomas Jefferson[d],"March 4, 1797","March 4, 1801"
3,Thomas Jefferson,1743,1826,"March 4, 1801–March 4, 1809",Democratic-Republican,1800 1804,Aaron Burr George Clinton,"March 4, 1801","March 4, 1809"
4,James Madison,1751,1836,"March 4, 1809–March 4, 1817",Democratic-Republican,1808 1812,"George Clinton[e] Vacant afterApril 20, 1812 E...","March 4, 1809","March 4, 1817"
5,James Monroe,1758,1831,"March 4, 1817–March 4, 1825",Democratic-Republican,1816 1820,Daniel D. Tompkins,"March 4, 1817","March 4, 1825"
6,John Quincy Adams,1767,1848,"March 4, 1825–March 4, 1829",Democratic-Republican[f] National Republican,1824,John C. Calhoun[g],"March 4, 1825","March 4, 1829"
7,Andrew Jackson,1767,1845,"March 4, 1829–March 4, 1837",Democratic,1828 1832,"John C. Calhoun[h] Vacant afterDecember 28, 18...","March 4, 1829","March 4, 1837"
8,Martin Van Buren,1782,1862,"March 4, 1837–March 4, 1841",Democratic,1836,Richard Mentor Johnson,"March 4, 1837","March 4, 1841"
9,William Henry Harrison,1773,1841,"March 4, 1841–April 4, 1841[e]",Whig,1840,John Tyler,"March 4, 1841","April 4, 1841[e]"


Мы разделили даты, но в некоторых из них существуют сноски из html страницы. Поэтому проделаем еще работу для того, чтобы сделать данные чище. 

In [518]:
df_USA_presidents['start'] = df_USA_presidents['start'].apply(lambda date: date.split('[')[0])
df_USA_presidents['end'] = df_USA_presidents['end'].apply(lambda date: date.split('[')[0])

In [520]:
df_USA_presidents.head()

,Name,Birth_date,Death_date,Term,Party,Election,Vice President,start,end
No,,,,,,,,,
1,George Washington,1732,1799,"April 30, 1789–March 4, 1797",Unaffiliated,1788–1789 1792,John Adams[c],"April 30, 1789","March 4, 1797"
2,John Adams,1735,1826,"March 4, 1797–March 4, 1801",Federalist,1796,Thomas Jefferson[d],"March 4, 1797","March 4, 1801"
3,Thomas Jefferson,1743,1826,"March 4, 1801–March 4, 1809",Democratic-Republican,1800 1804,Aaron Burr George Clinton,"March 4, 1801","March 4, 1809"
4,James Madison,1751,1836,"March 4, 1809–March 4, 1817",Democratic-Republican,1808 1812,"George Clinton[e] Vacant afterApril 20, 1812 E...","March 4, 1809","March 4, 1817"
5,James Monroe,1758,1831,"March 4, 1817–March 4, 1825",Democratic-Republican,1816 1820,Daniel D. Tompkins,"March 4, 1817","March 4, 1825"


Так же сноски из html страницы есть в колонке party. Давайте почистим и эту колонку тоже.

In [521]:
df_USA_presidents['Party'] = df_USA_presidents['Party'].\
                    apply(lambda party: party.split('[')[0] + party.split('[')[1][2:] if '[' in party else party) 

In [522]:
df_USA_presidents

,Name,Birth_date,Death_date,Term,Party,Election,Vice President,start,end
No,,,,,,,,,
1,George Washington,1732,1799,"April 30, 1789–March 4, 1797",Unaffiliated,1788–1789 1792,John Adams[c],"April 30, 1789","March 4, 1797"
2,John Adams,1735,1826,"March 4, 1797–March 4, 1801",Federalist,1796,Thomas Jefferson[d],"March 4, 1797","March 4, 1801"
3,Thomas Jefferson,1743,1826,"March 4, 1801–March 4, 1809",Democratic-Republican,1800 1804,Aaron Burr George Clinton,"March 4, 1801","March 4, 1809"
4,James Madison,1751,1836,"March 4, 1809–March 4, 1817",Democratic-Republican,1808 1812,"George Clinton[e] Vacant afterApril 20, 1812 E...","March 4, 1809","March 4, 1817"
5,James Monroe,1758,1831,"March 4, 1817–March 4, 1825",Democratic-Republican,1816 1820,Daniel D. Tompkins,"March 4, 1817","March 4, 1825"
6,John Quincy Adams,1767,1848,"March 4, 1825–March 4, 1829",Democratic-Republican National Republican,1824,John C. Calhoun[g],"March 4, 1825","March 4, 1829"
7,Andrew Jackson,1767,1845,"March 4, 1829–March 4, 1837",Democratic,1828 1832,"John C. Calhoun[h] Vacant afterDecember 28, 18...","March 4, 1829","March 4, 1837"
8,Martin Van Buren,1782,1862,"March 4, 1837–March 4, 1841",Democratic,1836,Richard Mentor Johnson,"March 4, 1837","March 4, 1841"
9,William Henry Harrison,1773,1841,"March 4, 1841–April 4, 1841[e]",Whig,1840,John Tyler,"March 4, 1841","April 4, 1841"


Теперь займемся вычислением количества дней.

In [523]:
df_USA_presidents['start'] = pd.to_datetime(df_USA_presidents['start'])
df_USA_presidents['end'] = pd.to_datetime(df_USA_presidents['end'])


In [524]:
df_USA_presidents.head()

,Name,Birth_date,Death_date,Term,Party,Election,Vice President,start,end
No,,,,,,,,,
1,George Washington,1732,1799,"April 30, 1789–March 4, 1797",Unaffiliated,1788–1789 1792,John Adams[c],1789-04-30,1797-03-04
2,John Adams,1735,1826,"March 4, 1797–March 4, 1801",Federalist,1796,Thomas Jefferson[d],1797-03-04,1801-03-04
3,Thomas Jefferson,1743,1826,"March 4, 1801–March 4, 1809",Democratic-Republican,1800 1804,Aaron Burr George Clinton,1801-03-04,1809-03-04
4,James Madison,1751,1836,"March 4, 1809–March 4, 1817",Democratic-Republican,1808 1812,"George Clinton[e] Vacant afterApril 20, 1812 E...",1809-03-04,1817-03-04
5,James Monroe,1758,1831,"March 4, 1817–March 4, 1825",Democratic-Republican,1816 1820,Daniel D. Tompkins,1817-03-04,1825-03-04


In [525]:
df_USA_presidents['Term_in_days'] = (df_USA_presidents['end'] - df_USA_presidents['start']).dt.days

In [527]:
df_USA_presidents.head()

,Name,Birth_date,Death_date,Term,Party,Election,Vice President,start,end,Term_in_days
No,,,,,,,,,,
1,George Washington,1732,1799,"April 30, 1789–March 4, 1797",Unaffiliated,1788–1789 1792,John Adams[c],1789-04-30,1797-03-04,2865
2,John Adams,1735,1826,"March 4, 1797–March 4, 1801",Federalist,1796,Thomas Jefferson[d],1797-03-04,1801-03-04,1460
3,Thomas Jefferson,1743,1826,"March 4, 1801–March 4, 1809",Democratic-Republican,1800 1804,Aaron Burr George Clinton,1801-03-04,1809-03-04,2922
4,James Madison,1751,1836,"March 4, 1809–March 4, 1817",Democratic-Republican,1808 1812,"George Clinton[e] Vacant afterApril 20, 1812 E...",1809-03-04,1817-03-04,2922
5,James Monroe,1758,1831,"March 4, 1817–March 4, 1825",Democratic-Republican,1816 1820,Daniel D. Tompkins,1817-03-04,1825-03-04,2922


Мы вычислили срок правления в днях, теперь составим конечный вариант датафрейма. 

Мы будем использовать колонки `Name` - Имя, `Birth_date` - Дата рождения, `Death_date` - Дата смерти, `Term_in_days` - Срок правления в днях, `Party` - Партия. Колонки `Election` и `Vice President` в итоговый датафрейм мы не будем включать, так как в иследовании эти колонки не будут принимать участия.

In [528]:
df_USA_presidents = df_USA_presidents[['Name', 'Birth_date', 'Death_date', 'Term_in_days', 'Party']]

In [529]:
df_USA_presidents.head()

,Name,Birth_date,Death_date,Term_in_days,Party
No,,,,,
1,George Washington,1732,1799,2865,Unaffiliated
2,John Adams,1735,1826,1460,Federalist
3,Thomas Jefferson,1743,1826,2922,Democratic-Republican
4,James Madison,1751,1836,2922,Democratic-Republican
5,James Monroe,1758,1831,2922,Democratic-Republican


Итак мы провели подготовку и очистку данных и теперь можем с ними работать.

#### Краткая информация о датафрейме:

In [530]:
df_USA_presidents.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 1 to 45
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          45 non-null     object
 1   Birth_date    45 non-null     object
 2   Death_date    45 non-null     object
 3   Term_in_days  45 non-null     int64 
 4   Party         45 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.1+ KB


df_sales — результат запроса. В нём содержится следующая информация:

- Name - ФИО президента
- Birth_date - Дата рождения
- Death_date - Дата смерти (у бывших президентов, которые ныне живы, проставлены в строках 0)
- Term_in_days - Срок правления в днях
- Party - Партия президента

Так же в таблице исключен нынешний президент.

Пропуски не обнаружены. Необходимости преобразовывать типы нет.

Для срока правления в днях устраивает int64 - целые числа. Можно немного уменьшить занимаемую память, понизив тип до int16.


###### Уменьшим использование памяти.

In [531]:
df_USA_presidents = df_USA_presidents.astype({'Term_in_days' : 'int16'})

In [532]:
df_USA_presidents.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45 entries, 1 to 45
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          45 non-null     object
 1   Birth_date    45 non-null     object
 2   Death_date    45 non-null     object
 3   Term_in_days  45 non-null     int16 
 4   Party         45 non-null     object
dtypes: int16(1), object(4)
memory usage: 1.8+ KB


Незначительно уменьшели занимаемую память.

### Перейдем к анализу данных датафрейма с помощью SQLlite3.

Для этого создадим подключение к базе данных и подключим датафрейм к ней.

На выходе получаем количество строк в таблице.

In [539]:
con = sqlite3.connect('db')
df_USA_presidents_SQL = df_USA_presidents
df_USA_presidents_SQL.to_sql('USA_presidents_SQL',con,index=False,if_exists='replace')

45

Создадим функцию для того, чтобы удобнее передавать запросы к базе данных.

In [540]:
def select(sql):
    return pd.read_sql(sql,con)

##### Для начала выведем первых 10-ти президентов из таблицы.

In [541]:
sql = '''
SELECT 
    * 
FROM 
    USA_presidents_SQL
LIMIT 10
'''

In [542]:
select(sql)

,Name,Birth_date,Death_date,Term_in_days,Party
0,George Washington,1732,1799,2865,Unaffiliated
1,John Adams,1735,1826,1460,Federalist
2,Thomas Jefferson,1743,1826,2922,Democratic-Republican
3,James Madison,1751,1836,2922,Democratic-Republican
4,James Monroe,1758,1831,2922,Democratic-Republican
5,John Quincy Adams,1767,1848,1461,Democratic-Republican National Republican
6,Andrew Jackson,1767,1845,2922,Democratic
7,Martin Van Buren,1782,1862,1461,Democratic
8,William Henry Harrison,1773,1841,31,Whig
9,John Tyler,1790,1862,1430,Whig Unaffiliated


##### Далее давайте посчитаем сколько президентов от какой партии выступали.

In [543]:
sql = '''
SELECT 
    Party,
    COUNT(1) as Cnt_party
FROM 
    USA_presidents_SQL
GROUP BY 
    Party
ORDER BY 
    Cnt_party DESC
'''

In [544]:
select(sql)

,Party,Cnt_party
0,Republican,18
1,Democratic,15
2,Whig,3
3,Democratic-Republican,3
4,Whig Unaffiliated,1
5,Unaffiliated,1
6,Republican National Union,1
7,National Union Democratic,1
8,Federalist,1
9,Democratic-Republican National Republican,1


 Заметим, что большинство президентов выступали от партий - Republican и Democratic.

#### Взглянем на президента, которые дольше всех находился у правления страной.

In [545]:
sql = '''
SELECT 
    Name,
    Term_in_days
FROM 
    USA_presidents_SQL
WHERE 
    Term_in_days = (SELECT MAX(Term_in_days) FROM USA_presidents_SQL)
'''

In [546]:
select(sql)

,Name,Term_in_days
0,Franklin D. Roosevelt,4422


In [548]:
4422 / 365.5

12.098495212038303

В результате запроса мы выяснили, что дольше всех у правления был Franklin D. Roosevelt, который занимал место президента больше 12-ти лет.

#### Теперь посмотрим  на президента, которые меньше всех был у власти.

In [549]:
sql = '''
SELECT 
    Name,
    Term_in_days
FROM 
    USA_presidents_SQL
WHERE 
    Term_in_days = (SELECT MIN(Term_in_days) FROM USA_presidents_SQL)
'''

In [550]:
select(sql)

,Name,Term_in_days
0,William Henry Harrison,31


Данный запрос показал, что меньше всех был у власти William Henry Harrison, который умер спустя месяц после выборов.

####  Найдем количество президентов родившихся в 18, 19, 20 веках (Не считая нынешнего).

In [551]:
sql = '''
SELECT 
    COUNT(Name) FILTER(WHERE Birth_date BETWEEN 1700 and 1799) as '18th_century',
    COUNT(Name) FILTER(WHERE Birth_date BETWEEN 1800 and 1899) as '19th_century',
    COUNT(Name) FILTER(WHERE Birth_date BETWEEN 1900 and 1999) as '20th_century'
FROM 
    USA_presidents_SQL
'''

In [552]:
select(sql)

,18th_century,19th_century,20th_century
0,13,21,11


По выводу мы можем однозначно можем сказать:

    - 13 президентов родились в 18 веке;
    - 21 президент родились в 19 веке;
    - 11 президентов родились в 20 веке.

#### Вспомним как выглядел наша таблица и посмотрим на президента, который прожил больше других.

In [553]:
sql = '''
SELECT 
    * 
FROM 
    USA_presidents_SQL
LIMIT 
    5
'''

In [554]:
select(sql)

,Name,Birth_date,Death_date,Term_in_days,Party
0,George Washington,1732,1799,2865,Unaffiliated
1,John Adams,1735,1826,1460,Federalist
2,Thomas Jefferson,1743,1826,2922,Democratic-Republican
3,James Madison,1751,1836,2922,Democratic-Republican
4,James Monroe,1758,1831,2922,Democratic-Republican


In [555]:
sql = '''
SELECT 
    Name,
    MAX(Death_date - Birth_date) FILTER(WHERE Death_date != 0) as age
FROM 
    USA_presidents_SQL
'''

In [556]:
select(sql)

,Name,age
0,George H. W. Bush,94


Среди ранее живших George H.W. Bush - 41-ый президент США, прожил дольше остальных, но давайте проверим, есть ли кто-то, кто прожил больше лет и все еще живет свою прекрасную жизнь.

In [557]:
sql = '''
SELECT 
    Name,
    MAX(date('now') - Birth_date) FILTER(WHERE Death_date == 0) - 1 as age
FROM 
    USA_presidents_SQL
'''

In [558]:
select(sql)

,Name,age
0,Jimmy Carter,98


И к большому удивлению запрос вернул Jimmy Carter - 39-го президента США, который прожил на данный момент 98 лет.

## Выводы:

- Из страницы Википедии о презедентах США -  https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States мы достали данные с помощью Pandas и загрузили их в датафрейм.

- Поскольку данные нуждались в очистке и преобразовании, было решено произвести необходимые действия. 
    - Колонка с именем, датой рождения и смерти была разделена на три колонки, для более удобного взаимодействия с данными.
    - Было посчитано из дат о сроке правления, сколько дней пробыл каждый из презедентов на посту.
    - Данные были очищены от сносок, которые были полезны на html странице, но в нашей таблице не несли никакого смысла.
    
- В анализе данных с помощью подготовленного датафрейма был использован SQLite3 и проведены следующие действия:
    - Сколько выступало президентов от каждой партии. Больше всего президентов было от партий - Republican и Democratic.
    - Был найден президент, чей срок правления составлял больше остальных - Franklin D. Roosevelt, который занимал пост президента почти 13 лет. А наименьший срок имел - William Henry Harrison, который умер спустя месяц после выборов.
    - Так же был найден президент, который прожил среди ранее живших больше остальных - George H.W. Bush - 41-ый президент США. А при ныне живущих - Jimmy Carter - 39-ый президент США, который прожил на данный момент 98 лет.
    